In [135]:
import pandas as pd
import numpy as np
import os
import gzip
import pickle
import re
import copy
from tqdm import tqdm
import time

In [136]:
# Start,End,Open,High,Low,Close,Volume,Market Cap
# 2024-03-08,2024-03-09,66871.2,69451.15,66398.03,68202.87,71117263448.78049,1329444482209.993
price = pd.read_csv('../../pricehistory/price_full.csv') 

# Timestamp Graph,Intervals (Row),Network hashrate (TH/s),Bitcoins per block (BTC)
# 02/01/2009,1,4.97E-08,50
BitcoinData = pd.read_csv('BitcoinData.csv') 

# Date,ln(P_max),Data monthly average,Model ln(P_eff)
# 2011-04-01,-12.485786548332777,1.6612383008356546e-06,1.5657608695648479e-06
efficiency = pd.read_csv('../7_plotting/model_and_monthly_data_2.csv') 

df1 = pd.DataFrame(columns=['date', 'price'])
df2 = pd.DataFrame(columns=['date', 'hashrate','block'])
df3 = pd.DataFrame(columns=['date', 'efficiency'])


df1['date'] = pd.to_datetime(price['Start'])
df1['price'] = price['Close']

df2['date'] = pd.to_datetime(BitcoinData['Timestamp Graph'], format='%d/%m/%Y')
df2['hashrate'] = BitcoinData['Network hashrate (TH/s)']*1e12
df2['block'] = BitcoinData['Bitcoins per block (BTC)']

df3['date'] = pd.to_datetime(efficiency['Date'], format='%Y-%m-%d')
df3['efficiency'] = efficiency['Data monthly average']*1e12

joined = pd.merge(df1, df2, on='date', how='inner')
joined = pd.merge(joined, df3, on='date', how='inner')
joined = joined.dropna()

In [137]:
pd.set_option('display.max_rows', None)
joined.sample(10)

,date,price,hashrate,block,efficiency
812,2016-05-23,444.280000,1.750000e+18,25.00,3.027406e+09
929,2015-06-07,223.390000,3.950000e+17,25.00,1.383407e+09
1167,2013-06-23,107.600000,1.690000e+14,25.00,3.460336e+08
590,2018-03-19,8588.661986,2.190000e+19,12.50,8.429202e+09
1367,2011-10-31,3.248000,8.730000e+12,50.00,2.169418e+06
567,2018-05-28,7149.250597,3.340000e+19,12.50,8.884608e+09
554,2018-07-06,6600.606901,4.270000e+19,12.50,9.251122e+09
1309,2012-04-23,4.960000,1.160000e+13,25.00,4.156520e+06
242,2021-01-26,32577.768697,1.470000e+20,6.25,1.359253e+10
384,2019-11-27,7502.033156,1.050000e+20,12.50,1.244366e+10


In [138]:
joined["reward"] = joined["block"]*joined["price"]

data = []

for i in range(len(joined)):
    date = joined['date'].iloc[i]
    hashrate = joined['hashrate'].iloc[i]
    efficiency = joined['efficiency'].iloc[i]
    reward = joined['reward'].iloc[i]
    aux = 2.0
    electricity_cost = 0.05/(1000*3600)
    constant = 6
    watts = hashrate/efficiency
    top = watts*aux*electricity_cost
    reward_per_second = reward/600
    bottom = constant*reward_per_second
    cost = top/bottom
    data.append([date, cost])
    
    print(f"date: {date}")
    # print(f"hashrate: {hashrate}")
    # print(f"efficiency: {efficiency}")
    print(f"watts: {watts}")
    print(f"top: {top}")
    print(f"reward: {reward}")
    print(f"bottom: {bottom}")
    print(f"cost: {cost}")
    print("\n\n")

df = pd.DataFrame(data, columns=['date', 'cost'])

date: 2022-06-30 00:00:00
watts: 14783167354.34259
top: 410.64353762062757
reward: 123854.66474956086
bottom: 1238.5466474956088
cost: 0.33155274244289895



date: 2022-06-27 00:00:00
watts: 11779416218.599674
top: 327.2060060722132
reward: 129581.14625
bottom: 1295.8114625
cost: 0.25251050445327666



date: 2022-06-24 00:00:00
watts: 11190445407.66969
top: 310.8457057686025
reward: 132997.0770573619
bottom: 1329.9707705736191
cost: 0.23372371231477074



date: 2022-06-21 00:00:00
watts: 13310740327.017632
top: 369.7427868616009
reward: 129624.64616074506
bottom: 1296.2464616074506
cost: 0.2852411156463948



date: 2022-06-18 00:00:00
watts: 13840814056.854616
top: 384.4670571348505
reward: 118796.894375
bottom: 1187.96894375
cost: 0.3236339292854098



date: 2022-06-15 00:00:00
watts: 14606476111.063595
top: 405.73544752954433
reward: 140566.19204571532
bottom: 1405.6619204571532
cost: 0.2886436927860932



date: 2022-06-12 00:00:00
watts: 11249342488.762688
top: 312.4817357989636
rew

In [139]:
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
# from datetime import timedelta
# df['avg'] = (df['date'] - timedelta(days=15)).dt.to_period('Q')
df.sample(10)

,date,cost
533,2018-02-11,0.077144
516,2018-04-04,0.093965
579,2017-09-27,0.070671
734,2016-06-19,0.083175
1397,2011-01-07,0.017297
1039,2013-12-16,0.002164
294,2020-01-29,0.197194
643,2017-03-18,0.156890
662,2017-01-20,0.131539
116,2021-07-17,0.082418


In [140]:
# avg_data = df.groupby('avg')['cost'].mean().reset_index()
# avg_data

In [141]:
df.to_csv('cost.csv', index=False)